# 自动微分
:label:`sec_autograd`

正如 :numref:`sec_calculus`中所说，求导是几乎所有深度学习优化算法的关键步骤。
虽然求导的计算很简单，只需要一些基本的微积分。
但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。

深度学习框架通过自动计算导数，即*自动微分*（automatic differentiation）来加快求导。
实际中，根据设计好的模型，系统会构建一个*计算图*（computational graph），
来跟踪计算是哪些数据通过哪些操作组合起来产生输出。
自动微分使系统能够随后反向传播梯度。
这里，*反向传播*（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

## 一个简单的例子

作为一个演示例子，(**假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**)。
首先，我们创建变量`x`并为其分配一个初始值。


In [ ]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度。**]
重要的是，我们不会在每次对一个参数求导时都分配新的内存。
因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。


In [ ]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

(**现在计算$y$。**)


In [ ]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

`x`是一个长度为4的向量，计算`x`和`x`的点积，得到了我们赋值给`y`的标量输出。
接下来，[**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**]，并打印这些梯度。


In [ ]:
y.backward()  #通过调用反向传播函数来自动计算y关于x每个分量的梯度
x.grad

tensor([ 0.,  4.,  8., 12.])

函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于$\mathbf{x}$的梯度应为$4\mathbf{x}$。
让我们快速验证这个梯度是否计算正确。


In [ ]:
x.grad == 4 * x

tensor([True, True, True, True])

[**现在计算`x`的另一个函数。**]


In [ ]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()  #将梯度归零
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播

当`y`不是标量时，向量`y`关于向量`x`的导数的最自然解释是一个矩阵。
对于高阶和高维的`y`和`x`，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括[**深度学习中**]），
但当调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
这里(**，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。**)


In [ ]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 分离计算

有时，我们希望[**将某些计算移动到记录的计算图之外**]。
例如，假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。
想象一下，我们想计算`z`关于`x`的梯度，但由于某种原因，希望将`y`视为一个常数，
并且只考虑到`x`在`y`被计算后发挥的作用。

这里可以分离`y`来返回一个新变量`u`，该变量与`y`具有相同的值，
但丢弃计算图中如何计算`y`的任何信息。
换句话说，梯度不会向后流经`u`到`x`。
因此，下面的反向传播函数计算`z=u*x`关于`x`的偏导数，同时将`u`作为常数处理，
而不是`z=x*x*x`关于`x`的偏导数。


In [ ]:
x.grad.zero_()
y = x * x
u = y.detach()  #将某些计算移动到记录的计算图之外
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

由于记录了`y`的计算结果，我们可以随后在`y`上调用反向传播，
得到`y=x*x`关于的`x`的导数，即`2*x`。


In [ ]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度计算

使用自动微分的一个好处是：
[**即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**]。
在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [ ]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

让我们计算梯度。


In [ ]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()  #即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

我们现在可以分析上面定义的`f`函数。
请注意，它在其输入`a`中是分段线性的。
换言之，对于任何`a`，存在某个常量标量`k`，使得`f(a)=k*a`，其中`k`的值取决于输入`a`，因此可以用`d/a`验证梯度是否正确。


In [ ]:
a.grad == d / a

tensor(True)

## 小结

* 深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上，然后记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。

## 练习

1. 为什么计算二阶导数比一阶导数的开销要更大？
1. 在运行反向传播函数之后，立即再次运行它，看看会发生什么。
1. 在控制流的例子中，我们计算`d`关于`a`的导数，如果将变量`a`更改为随机向量或矩阵，会发生什么？
1. 重新设计一个求控制流梯度的例子，运行并分析结果。
1. 使$f(x)=\sin(x)$，绘制$f(x)$和$\frac{df(x)}{dx}$的图像，其中后者不使用$f'(x)=\cos(x)$。


In [ ]:
'''
1.为什么计算二阶导数比一阶导数的开销要更大？
计算二阶导数（Hessian）比计算一阶导数（梯度）的开销大主要有以下几个原因：
 复杂性：计算二阶导数涉及到对梯度向量中的每个分量进行偏导数的计算。这意味着在 n 维空间中，计算 Hessian 需要 n^2 个偏导数，而计算梯度只需要 n 个偏导数。
 存储和计算成本：Hessian 矩阵通常是一个 n×n 的矩阵，而梯度只需一个 n 维向量。这在内存和计算时间上都会增加开销。
 数值稳定性：在计算二阶导数时，数值稳定性可能会受到挑战，特别是在使用数值方法进行逼近时，可能会导致额外的误差。

2.反向传播函数的重复运行会发生什么？
在深度学习框架（如 TensorFlow 或 PyTorch）中，当调用反向传播函数后，梯度会被计算并存储（通常是通过链式法则）。如果立即再次运行反向传播函数：
 梯度会累加：在许多框架中，如果不清除梯度，梯度会在每次反向传播时累加。这样可能导致最终梯度的计算不是我们想要的结果。
 需要重置：通常需要在每次反向传播之前调用 zero_grad()，以清除先前的梯度。

3.控制流与随机变量或矩阵的导数
如果您计算某个函数 d 关于一个变量 a 的导数，但将 a 更改为一个随机向量或矩阵，可能会发生以下情况：
 误差传播：如果控制流或分支条件依赖于 a 的值，随机化可能会导致程序执行的分支变化，从而使梯度的计算无意义。
 维度不匹配：随机向量或矩阵的使用可能会导致维度不一致，特别是在计算梯度或 Hessian 时。

4.重新设计控制流梯度的示例
 下面是一个简单的示例，其中我们计算函数 f(a)=a^2 的导数，使用控制流来处理条件：
'''
import torch
# 控制流示例
def control_flow_example(a):
    if a.item() > 0:
        return a ** 2
    else:
        return -a ** 2

# 随机向量
a = torch.tensor(1.0, requires_grad=True)
output = control_flow_example(a)
output.backward()
print("Gradient with a = 1.0:", a.grad)

a = torch.tensor(-1.0, requires_grad=True)
output = control_flow_example(a)
output.backward()
print("Gradient with a = -1.0:", a.grad)

'''
5. 画出 f(x)=sin⁡(x) 的图像以及其导数
我们将在不使用已知导数的方法 f'(x)=cos⁡(x) 的情况下，通过数值方法计算并绘制。
以下是 Python 代码：
'''
import numpy as np
import matplotlib.pyplot as plt

# 定义函数
def f(x):
    return np.sin(x)

# 数值计算导数
def numerical_derivative(f, x, h=1e-5):
    return (f(x + h) - f(x - h)) / (2 * h)

# 定义 x 的取值范围
x_values = np.linspace(-2 * np.pi, 2 * np.pi, 100)
f_values = f(x_values)
df_values = numerical_derivative(f, x_values)

# 绘制函数和导数
plt.figure(figsize=(12, 6))
plt.plot(x_values, f_values, label='f(x) = sin(x)', color='blue')
plt.plot(x_values, df_values, label="Numerical df(x)/dx", linestyle='--', color='orange')
plt.title('Function f(x) and its Numerical Derivative')
plt.xlabel('x')
plt.ylabel('y')
plt.axhline(0, color='black', linewidth=0.5, ls='--')
plt.axvline(0, color='black', linewidth=0.5, ls='--')
plt.legend()
plt.grid()
plt.show()

'''
总结
计算开销：二阶导数计算比一阶导数更复杂且开销更大。
反向传播：多次运行期间梯度可累加，需注意重置。
随机变量：修改变量可能导致梯度计算出错。
控制流：控制流下的导数计算可以具有挑战性，变化值会影响结果。
数值导数：通过差分法计算导数，而不仅依赖解析解。
'''

[Discussions](https://discuss.d2l.ai/t/1759)
